In [1]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer

## 분석에 필요한 다양한 라이브러리를 import합니다.
1. pandas: 데이터를 빠르고 강력하게, 그리고 유연하고 쉽게 다룰 수 있게 해주는 데이터 분석, 증강 도구입니다.
    - <https://pandas.pydata.org/docs/user_guide/index.html>

2. numpy: 파이썬에서 수치 계산을 빠르게 수행할 수 있게 해주는 도구입니다.
    - <https://numpy.org/doc/stable/>


3. matplolib: 정적, 애니메이션, 또는 상호작용형 시각화를 생성하기 위한 포괄적인 도구입니다.
    - <https://matplotlib.org/stable/users/index>


4. scikit-learn: 데이터 분석을 위한, 쉽고 효율적인 여러 도구를 제공합니다.
    - <https://scikit-learn.org/stable/user_guide.html>

In [2]:
# 데이터가 없어서 일단은 유방암 데이터로 진행(나중에 승현님한테서 데이터 받아 대체해야함)
breast_cancer = load_breast_cancer()
df = pd.DataFrame(data = breast_cancer.data, columns = breast_cancer.feature_names)
df

# 데이터프레임에서 10개의 열만 사용
df = df.iloc[:, :10]

# 타겟 데이터를 가져와서 데이터프레임에 label이라는 열로 추가
df["label"] = breast_cancer.target
df.columns = [ col.replace(" ", "_") for col in df.columns]

# 확인
df.head(3)
df

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,label
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0
...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,0


데이터프레임의 전체적인 모습은 위와 같습니다.

0번째 열부터 166번째 열까지는 독립변수이고, 마지막 열은 반응변수(정답값)입니다.

1. 166개의 독립변수는 smiles 형식으로 표현된 화학식을 footprint로 변환한 결과입니다.
2. 반응변수는 NR-AR입니다.
    - 12가지의 독성 분석 중 NR-AR를 선택해, 독성 유무를 예측합니다.

In [6]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

데이터프레임을 훈련 데이터와 테스트 데이터로 나눕니다.

- 독립 변수는 마지막 열을 제외한 모든 열입니다.
- 종속 변수는 마지막 열입니다.

In [11]:
from sklearn.linear_model import LogisticRegression


# 로지스틱 회귀 모델 생성
model = LogisticRegression(penalty='l2', C=1.0, solver='liblinear', class_weight=None)
model.fit(X, y)

LogisticRegression(solver='liblinear')

## 모델을 생성합니다.

선택한 모델은 로지스틱 회귀 모델입니다. 
- 로지스틱 회귀 모델은 선형 회귀 모델을 기반으로 하되, 시그모이드 함수를 사용하여 0과 1 사이의 값을 출력하는 모델입니다. 이 모델은 이진 분류 문제에 주로 사용됩니다.
전처리를 수행한 데이터에 대해서 로지스틱 회귀 모델을 학습시키고, 학습된 모델을 사용하여 예측을 수행해보겠습니다.

## 로지스틱 회귀 모형은 아래의 하이퍼파라미터를 조정할 수 있습니다.

$$
J(\theta) = -\frac{1}{m}\sum_{i=1}^{m} [y^{(i)}\log(h_{\theta}(x^{(i)})) + (1-y^{(i)})\log(1-h_{\theta}(x^{(i)}))] + \frac{\lambda}{2m}\sum_{j=1}^{n}\theta_{j}^2
$$

1. penalty: 규제의 종류를 설정합니다. 기본값은 'l2'입니다.
    - 'l2': L2 규제를 의미합니다.
    - 위의 식에서 마지막 항이 L2 규제 부분입니다.
2. C: 규제의 강도를 설정합니다. 기본값은 1.0입니다.
    - C가 작을수록 규제가 강해집니다.

3. solver: 최적화에 사용할 알고리즘을 설정합니다. 기본값은 'lbfgs'입니다.
    - 'newton-cg': Netwon-Conjugate Gradient 알고리즘입니다.
    - 'lbfgs': L-BFGS 알고리즘입니다.
    - 'liblinear': 라이브러리의 솔버입니다.
    - 'sag': Stochastic Average Gradient 알고리즘입니다.
    - 'saga': SAGA 알고리즘입니다.
    - 각 알고리즘 별로 장단점이 있으며, 데이터의 크기에 따라 성능이 달라질 수 있습니다.

4. class_weight: 클래스의 가중치를 설정합니다. 기본값은 None입니다.
    - None: 클래스의 가중치를 동일하게 설정합니다.
    - 'balanced': 클래스의 빈도에 반비례하게 가중치를 설정합니다.
    - 혹은 사용자가 직접 클래스의 가중치를 설정할 수 있습니다.

In [12]:
# 혼동 행렬
from sklearn.metrics import confusion_matrix
confusion_matrix(y, model.predict(X))

array([[176,  36],
       [ 16, 341]])

## 혼동 행렬을 출력합니다.

혼동행렬은 2x2 행렬로, 실제 클래스와 예측 클래스가 일치하는지 여부에 따라 4개의 값을 가집니다.
- 182개의 양성 샘플을 양성으로 정확하게 예측했습니다.
- 344개의 음성 샘플을 음성으로 정확하게 예측했습니다.
- 30개의 양성 샘플을 음성으로 잘못 예측했습니다.
- 11개의 음성 샘플을 양성으로 잘못 예측했습니다.

**나중에 실제 독성 데이터로 대체 후, 구체적인 값은 결과에 맞춰 대체해야 함**